# Introduction to stochastic optimization

The content in this class is based on the book "Uma Introducção à Otimização sob Incerteza" by Humberto Bortolossi and Bernardo Pagnoncelli.

## About stochastic optimization
    
Most real-life problems bring uncertainties in themselves: they are inherent in virtually all systems related to actuarial science, economics, meteorology, demography, ecology, and so on. Nowadays, problems involving interactions between man, nature and technology are subject to rapid change, which increases uncertainty. Each new technological revolution brings new challenges to the knowledge established up to then. Even in the deterministic context, there are systems that are complex, which do not allow precise measurement of their parameters.
   
The area of stochastic optimization (also known as optimization under uncertainty) studies methods to address such situations: they incorporate uncertainties in modeling through the inclusion of random variables of known probability distributions. The aim is to find solutions that are acceptable for all possible realizations of the modeled random variables.
   
## The farmer problem

### Deterministic model

John is a farmer with $500$ Hectares (ha) of land available for cultivation. Remember that $500 ha$ equals $5000000 m^2$. He is a specialist in three crops: wheat, maize, and sugarcane. During the winter, he has to decide how much land will be dedicated to each of the three cultures. The figure below shows two possible land divisions.

![Figure 1](Figure1.png)

In addition to the size of its property, John has other constraints to consider. He is also the owner of livestock, which needs to be fed. Their livestock needs at least 200 tons (T) of wheat and 240 tons of corn for the ration.  Aside from the wheat and corn produced on his land, he can buy these products from other producers in the local market. Their excess production can be sold to wholesalers, for the price and much smaller because of the profit margin of these traders.

Sugar cane is a crop exclusively for profit: all its production is sold to wholesalers at 36 dollars per tonne (dollars/T). However, the government imposes a production quota of 6,000 T: any quantity produced above this value should be sold for only 10 dollars/T.

Based on information from previous years, John knows that the average yield of his crops is 2.5, 3.0 and 20 tons per hectare (T/ha). In addition, there is a specific production cost of each crop, which is given in  dollars/ha. The complete model data are shown in the table below:

|                                .  | wheat | corn |      sugarcane     |
|:---------------------------------:|:-----:|:----:|:------------------:|
|            yield (T/ha)           |  2.5  |  3.0 |         20         |
|   production costs (dollars/ha)   |  150  |  230 |         260        |
|     selling price (dollars/T)     |  170  |  150 | 36 ($\leq$ 6000 T) |
|                        -          |   -   |   -  |    10 (> 6000 T)   |
|      buying price (dollars/T)     |  238  |  210 |         -          |
| minimum requirement for livestock |  200  |  240 |         -          |
| total available land: 500 ha|


Remember that the total available land is 500 ha.

To help John decide how to divide his lands in order to maximize his profits, let's formulate a linear optimization problem that describes this situation. Define:

|variable|Meaning                                 |
|--------|:---------------------------------------|
| $x_1$  | hectares dedicated to wheat            |
| $x_2$  | hectares dedicated to corn             |
| $x_3$  | hectares dedicated to sugarcane        |
| $w_1$  | tonnes of wheat sold                   |
| $y_1$  | tonnes of wheat bought                 |
| $w_2$  | tonnes of corn sold                    |
| $y_2$  | tonnes of corn bought                  |
| $w_3$  | tonnes of sugarcane sold at 36 dollars |
| $w_4$  | tonnes of sugarcane sold at 10 dollars |

With this variables we can write the linear programming problem as 

$$
\begin{align}
%
\max  170w_1 +150w_2 +36w_3 +10w_4 -150x_1 - 230 x_2 -&260 x_3 -
238y_1 -210y_2\\
%
\mbox{s.t.: } & \nonumber \\
x_1 + x_2 + x_3 &\leq 500,\\
2.5x_1 + y_1 - w_1 &\geq 200, \\
3x_2 + y_2 - w_2 &\geq 240, \\
w_3 + w_4 &\leq 20 x_3, \\
w_3 &\leq 6000, \\
x_1, x_2, x_3, y_1, y_2, w_1, w_2, w_3, w_4  &\geq 0\\
%
\end{align}
$$

And good news, we can solve this with JuMP. Make sure the `Project.toml` and `Manifest.toml` files are in the same repository as this notebook so we can intall all packages automatically.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Excellent! Everything is good to go!


In [2]:
using JuMP, GLPK

model = Model(with_optimizer(GLPK.Optimizer))

@variable(model, x[i in 1:3] >= 0)
@variable(model, y[i in 1:2] >= 0)
@variable(model, w[i in 1:4] >= 0)

@constraints(model, begin
    x[1] + x[2] + x[3] <= 500
    2.5*x[1] + y[1] - w[1] >= 200
    3*x[2] + y[2] - w[2] >= 240
    w[3] + w[4] <= 20*x[3]
    w[3] <= 6000
end)

@objective(model, Max, 170*w[1] + 150*w[2] + 36*w[3] + 10*w[4] - 150*x[1] - 230*x[2] - 260*x[3] - 238*y[1] - 210*y[2])
model

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1192
┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1192


A JuMP Model
Maximization problem with:
Variables: 9
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: w, x, y

In [3]:
optimize!(model)
println("Dedicated area $(JuMP.value.(x))")
println("Total produced $(JuMP.value.(x).*[2.5; 3.0; 20])")
println("Total sold $(JuMP.value.(w))")
println("Total bought $(JuMP.value.(y))")
println("Profit: $(JuMP.objective_value(model))")

Dedicated area [120.0, 80.0, 300.0]
Total produced [300.0, 240.0, 6000.0]
Total sold [100.0, 0.0, 6000.0, 0.0]
Total bought [0.0, 0.0]
Profit: 118600.0


The solution can be summarized in the table below

|      .       | wheat | corn | sugarcane      |
|--------------|-------|------|----------------|
| Area (ha)    | 120   | 80   | 300            |
| produced (T) | 300   | 240  | 6000           |
| sold (T)     | 100   | -    | 6000           |
| bought (T)   | -     | -    | -              |
| Profit: 118 600 dollars |

John is happy for a moment. The same night John found his optimal land division he became suspicious of the solution. What if his experience in relation to the average income of cultures is not accurate as he thinks? What if the year in question has a particularly unfavorable climate and its crop yields less than expected? In those cases would the same land division the best possible?

Let us suppose that in a particularly favorable year yields are 20% higher than the average yields suggested by John. The data for the problem becomes

|                                .  | wheat | corn |      sugarcane     |
|:---------------------------------:|:-----:|:----:|:------------------:|
|            yield (T/ha)           |  3    |  3.6 |         24         |
|   production costs (dollars/ha)   |  150  |  230 |         260        |
|     selling price (dollars/T)     |  170  |  150 | 36 ($\leq$ 6000 T) |
|                        -          |   -   |   -  |    10 (> 6000 T)   |
|      buying price (dollars/T)     |  238  |  210 |         -          |
| minimum requirement for livestock |  200  |  240 |         -          |
| total available land: 500 ha|

Try solving this problem yourself! The answers should match with the following table

|      .       | wheat | corn | sugarcane      |
|--------------|-------|------|----------------|
| Area (ha)    | 183.33| 66.67| 250            |
| produced (T) | 550   | 240  | 6000           |
| sold (T)     | 350   | -    | 6000           |
| bought (T)   | -     | -    | -              |
| Profit: 167 666 dollars |

In [4]:
using JuMP, GLPK

model = Model(with_optimizer(GLPK.Optimizer))

@variable(model, x[i in 1:3] >= 0)
@variable(model, y[i in 1:2] >= 0)
@variable(model, w[i in 1:4] >= 0)

@constraints(model, begin
    #TODO
end)

@objective(model, Max, 170*w[1] + 150*w[2] + 36*w[3] + 10*w[4] - 150*x[1] - 230*x[2] - 260*x[3] - 238*y[1] - 210*y[2])
model

A JuMP Model
Maximization problem with:
Variables: 9
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: w, x, y

In [5]:
optimize!(model)
println("Dedicated area $(JuMP.value.(x))")
println("Total produced $(JuMP.value.(x).*[3.0; 3.6; 24])")
println("Total sold $(JuMP.value.(w))")
println("Total bought $(JuMP.value.(y))")
println("Profit: $(JuMP.objective_value(model))")

Dedicated area [NaN, NaN, NaN]
Total produced [NaN, NaN, NaN]
Total sold [1.0, NaN, NaN, NaN]
Total bought [NaN, NaN]
Profit: 0.0


Let us suppose that in a particularly favorable year yields are 20% lower than the average yields suggested by John. The data for the problem becomes

|                                .  | wheat | corn |      sugarcane     |
|:---------------------------------:|:-----:|:----:|:------------------:|
|            yield (T/ha)           |  2    |  2.4 |         16         |
|   production costs (dollars/ha)   |  150  |  230 |         260        |
|     selling price (dollars/T)     |  170  |  150 | 36 ($\leq$ 6000 T) |
|                        -          |   -   |   -  |    10 (> 6000 T)   |
|      buying price (dollars/T)     |  238  |  210 |         -          |
| minimum requirement for livestock |  200  |  240 |         -          |
| total available land: 500 ha|

Try solving this problem yourself! The answers should match with the following table

|      .       | wheat | corn | sugarcane      |
|--------------|-------|------|----------------|
| Area (ha)    | 100   | 25   | 375            |
| produced (T) | 200   | 60   | 6000           |
| sold (T)     | -     | -    | 6000           |
| bought (T)   | -     | 180  | -              |
| Profit: 59 950 dollars |

In [6]:
using JuMP, GLPK

model = Model(with_optimizer(GLPK.Optimizer))

@variable(model, x[i in 1:3] >= 0)
@variable(model, y[i in 1:2] >= 0)
@variable(model, w[i in 1:4] >= 0)

@constraints(model, begin
    #TODO
end)

@objective(model, Max, 170*w[1] + 150*w[2] + 36*w[3] + 10*w[4] - 150*x[1] - 230*x[2] - 260*x[3] - 238*y[1] - 210*y[2])
model

A JuMP Model
Maximization problem with:
Variables: 9
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: w, x, y

In [7]:
optimize!(model)
println("Dedicated area $(JuMP.value.(x))")
println("Total produced $(JuMP.value.(x).*[2.0; 2.4; 16])")
println("Total sold $(JuMP.value.(w))")
println("Total bought $(JuMP.value.(y))")
println("Profit: $(JuMP.objective_value(model))")

Dedicated area [NaN, NaN, NaN]
Total produced [NaN, NaN, NaN]
Total sold [1.0, NaN, NaN, NaN]
Total bought [NaN, NaN]
Profit: 0.0


These results are alarming for John's finances: 20% changes in crop yields relative to average income make their profit range from 59,950 dollars to 167,667 dollars! Thinking about sugar cane, John has the following dilemma: If he reserves a large area for this crop and yields are above average, he will have to sell a quantity of the product to an unfavorable price because of the quota. On the other hand, if he reserves a very small area and the yields are below the average, then he will lose the opportunity to sell cane at a favorable price.

### Stochastic model

John concludes that there is no solution that is the best for all cases. However, he wonders if there is a solution that is satisfactory for all types of possible yields.

Let's introduce a bit of nomenclature: the scenarios 20% above average, on average and 20% below average will be indexed by $s = 1,2,3$ respectively. The variables y and w have the same meaning as before, but will be indexed by $w_{is}, i = 1,2,3,4, s = 1,2,3$ and $y_{js}, j = 1, 2, s = 1,2, 3$. For example, $y_{23}$ represents the quantity of corn sold in the case of prices below the average. Let's assume that the scenarios have the same probability, that is, that each occurs with probability 1/3. Furthermore, assuming that John wants to maximize his long-term gains, it is reasonable to assume that he seeks a solution that maximizes his expected profit.

The problem becomes:

$$
\begin{align}
%
\max  -150x_1 - 230 x_2 -260x_3&\\
    +\frac{1}{3}(170w_{11} +150w_{21} +36w_{31} +10w_{41} &-238y_{11} -210y_{21})\\
    +\frac{1}{3}(170w_{12} +150w_{22} +36w_{32} +10w_{42} &-238y_{12} -210y_{22})\\
    +\frac{1}{3}(170w_{13} +150w_{23} +36w_{33} +10w_{43} &-238y_{13} -210y_{23})\\
%
\mbox{s.t.: } \nonumber &\\
x_1 + x_2 + x_3 &\leq 500,\\
\\
s = 1\\
3x_1 + y_{11} - w_{11} &\geq 200, \\
3.6x_2 + y_{21} - w_{21} &\geq 240, \\
w_{31} + w_{41} &\leq 24 x_3, \\
w_{31} &\leq 6000, \\
s = 2\\
2.5x_1 + y_{12} - w_{12} &\geq 200, \\
3x_2 + y_{22} - w_{22} &\geq 240, \\
w_{32} + w_{42} &\leq 20 x_3, \\
w_{32} &\leq 6000, \\
s = 3\\
2x_1 + y_{13} - w_{13} &\geq 200, \\
2.4x_2 + y_{23} - w_{23} &\geq 240, \\
w_{33} + w_{43} &\leq 16 x_3, \\
w_{33} &\leq 6000, \\
x_1, x_2, x_3, y_{11}, y_{21}, y_{12}, y_{22}, y_{13}, y_{23},\\
w_{11}, w_{21}, w_{31}, w_{41}, w_{12}, w_{22}, w_{32}, w_{42}, w_{13}, w_{23}, w_{33}, w_{43},    &\geq 0\\
%
\end{align}
$$


This is the so-called extensive form of a stochastic optimization problem. This denomination comes from the fact that all the variables that depend on scenarios are explicitly described in the model. Variables $x$ are called first-stage variables, since their value has to be defined before the climate is known and, consequently, the yield of the crops. Variables $y$ and $w$ are the second-stage variables. These are the variables that are chosen after the knowledge of the yield of the crops. They serve to correct a possible situation of a deficit in livestock feeding needs resulting from the choice of the first stage. The farmer's problem is an example of a two-stage resource problem.

So now it is yout turn to help John decide the division of the land! The profit should be 108 390 dollars!

In [8]:
using JuMP, GLPK

model = Model(with_optimizer(GLPK.Optimizer))
# Variables

# Constraints

# Objetive
model

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [9]:
optimize!(model)
println("Dedicated area $(JuMP.value.(x))")
println("Total sold $(JuMP.value.(w))")
println("Total bought $(JuMP.value.(y))")
println("Profit: $(JuMP.objective_value(model))")

glp_add_cols: ncs = 0; invalid number of columns
Error detected in file api/prob1.c at line 362


GLPKFatalError: GLPKFatalError("GLPK call failed. All GLPK objects you defined so far are now invalidated.")

Now that you have the solution for the stochastic formulation you can evaluate some metrics to evaluate your stochastic model

### Expected Value of Perfect Information (EPVI)
Imagine that John has a crystal ball and can predict the weather in the future. Under this hypothesis, he does not need the statistical model whenever he foresees an income 20% below the average he chooses the solution given for the model that considers this scenario as unique. If yields are average or above the average, he can adapt the model. 
If we expect a large number of years, then the average income of John under perfect information (WS) will be:

$$
WS = \frac{59 950$ + 118 600$ + 167 666$}{3} = 115 405 $
$$

Note that we are assuming that the different scenarios occur at random with probability 1/3 each. This average income corresponds to the situation under perfect information, that is, to the case where John knows precisely what will happen in the future.

Unfortunately, the meteorologists and we know that this hypothesis is not realistic. Thus, over a period of, say, 20 years, the best John has to do and use the stock solution given by the stochastic model, obtaining an expected profit of 108 390 dollars. The difference between this value and profit in the case under perfect information $WS$ is the expected value of perfect information or EVPI:

$$
EVPI = 115 405 $ - 118 390 $ = 7 015$
$$

### Value of Stochastic Solution (VSS)


The VSS measures the gain in considering the statistical model rather than just basing the decision on average incomes. Think that John is a stubborn farmer: even though he knows that possible income variances can occur, he insists on dividing his land according to the average income situation (our deterministic model). The profit obtained from this policy is called the Expected Value Solution or EEV

How do you calculate it? It is simple: fix the distribution of land in the case of medium incomes, that is, calculate the solution of the first deterministic model in the $y$ and $w$ variables, taking $x_1 = 120, x_2 = 80$ and $x_3 = 300$ and the yields equal to 3.0 3.6 and 24 (for s = 1) and then 2.0 2.4 and 16 (for s = 3). Solutions will be at 55 120 dollars and 148 000 dollars respectively. Recalling that the solution is 118600 dollars in the case of average income and 108 390 dollars in the case of the stochastic model, we have

$$
EEV = \frac{55 120$ + 118 600$ + 148 000$}{3} = 107240 $ \\
VSS = 108 390 - 107 240 $ = 1 150$
$$

### A final comment

The concepts of EVPI and VSS are important because they quantify the value of the information and the gain in considering the stochastic formulation. In the EVPI case, it tells you how much it is worth to pay for perfect information. The VSS gives us access to how much we are gaining in considering the stochastic model rather than merely assuming that crop yields are always average.